In [2]:
import os
import random
import numpy as np
import pandas as pd
from one_stage.model import VGGNet, CustomResNet18
from tools.dataset_loader import get_dataset
from torch import nn
from tqdm import tqdm

In [10]:
EPOCH = 40

from maml.models.maml import MAML, make

enc_args = {"bn_args": {"track_running_stats": False, "n_episode": 4}}

model = make("resnet18", enc_args, "logistic", {"n_way": 5})
model

MAML(
  (encoder): ResNet18(
    (layer0): Sequential(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    )
    (relu): ReLU(inplace=True)
    (layer1): Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (relu): ReLU(inplace=True)
    )
    (layer2): Block(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [3]:
# 'tl': meta-train loss
# 'ta': meta-train accuracy
# 'vl': meta-val loss
# 'va': meta-val accuracy
aves_keys = ["tl", "ta", "vl", "va"]
trlog = dict()
for k in aves_keys:
    trlog[k] = []

trlog

{'tl': [], 'ta': [], 'vl': [], 'va': []}

In [7]:
for epoch in range(0, EPOCH + 1):
    aves = {k: AverageMeter() for k in aves_keys}

    # meta-train
    model.train()
    np.random.seed(epoch)